In [136]:
import requests
import json

In [146]:
urlListDialOuts = 'https://callapi.gravitel.ru/api/v1/listdialouts'
urlAmo = 'https://testcallapi.amocrm.ru'
urlAuthAmo = '/private/api/auth.php?type=json'
urlPipeAmo = '/api/v2/pipelines'
urlLeadAmo = '/api/v2/leads'

##"USER_LOGIN": "nd@gravitel.ru",
##"USER_HASH": "5e0db73b6bb4ddf46fcb4107090cffdcc5026961"

# Enter login & Hash

loginUserAmo = 'nd@gravitel.ru'#input("Enter LOGIN user: ")
hashUserAmo = '5e0db73b6bb4ddf46fcb4107090cffdcc5026961'#input("Enter HASH user: ")

# Setting parameters

headers = {'Content-Type': 'application/json'}
authKeyAmo = {'USER_LOGIN': loginUserAmo, 'USER_HASH': hashUserAmo}

# Connect with amoCRM POST

authAmo = requests.post(urlAmo + urlAuthAmo, headers=headers, json=authKeyAmo).json()

# Список сотрудников

staff = {'daniil': '2285464', 'ilshat': '5906116'}

# check login & Hash

if 'error_code' in authAmo['response'].keys():
    print(authAmo['response']['error'])

## urlListLeadsAmo = '/api/v2/leads'
#print(authAmo['response'].keys())
print (staff)

{'daniil': '2285464', 'ilshat': '5906116'}


In [147]:
# Получить все воронки и этапы, метод GET
'''
pipeKeyAmo = {'USER_LOGIN': loginUserAmo, 'USER_HASH': hashUserAmo, 'id': 3139387}
pipelines = requests.get(urlAmo + urlPipeAmo, headers=headers, params=pipeKeyAmo).json()

if 'title' in pipelines.keys():
    print(pipelines['detail'])


'''
#print(pipelines)

"\npipeKeyAmo = {'USER_LOGIN': loginUserAmo, 'USER_HASH': hashUserAmo, 'id': 3139387}\npipelines = requests.get(urlAmo + urlPipeAmo, headers=headers, params=pipeKeyAmo).json()\n\nif 'title' in pipelines.keys():\n    print(pipelines['detail'])\n\n\n"

In [187]:
# Получить все сделки, метод GET

leadKeyAmo = {'USER_LOGIN': loginUserAmo, 'USER_HASH': hashUserAmo}
leads = requests.get(urlAmo + urlLeadAmo, headers=headers, params=leadKeyAmo).json()

if 'title' in leads.keys():
    print(leads['detail'])
leads = leads['_embedded']['items']

leads = {leads[i]: leads[i+1] for i in range(0, len(leads), 2)}

#leads['_embedded']['items']
#dbCall = staff


TypeError: unhashable type: 'dict'

In [138]:
# Запись в файл
#json.dump(leads,open("leads.txt","w"))